1、定义工具

In [1]:
import os
import dotenv
from langchain_community.tools.tavily_search import TavilySearchResults

dotenv.load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

search = TavilySearchResults(max_results = 1)

res = search.invoke("2025年11月3日上海天气怎么样？")
print(res)

/tmp/ipykernel_3936413/2563159751.py:9: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results = 1)


[{'title': '冷空气“浅冻”一下就溜，上海本周三升温遇小雨 - 新浪财经', 'url': 'https://finance.sina.com.cn/jjxw/2025-11-03/doc-infwahzv5888719.shtml', 'content': '新浪首页\n 新闻\n 体育\n 财经\n 娱乐\n 科技\n 博客\n 图片\n 专栏\n 更多\n\n 汽车 教育 时尚 女性 星座 健康\n 房产历史视频收藏育儿读书\n 佛学游戏旅游邮箱导航\n\n 新浪微博\n 新浪新闻\n 新浪财经\n 新浪体育\n 新浪众测\n 新浪博客\n 新浪视频\n 新浪游戏\n 天气通\n\n经济新闻滚动 > 正文\n\n# 冷空气“浅冻”一下就溜，上海本周三升温遇小雨\n\n冷空气“浅冻”一下就溜，上海本周三升温遇小雨\n\n2025年11月03日 06:02 滚动播报\n\n新浪财经APP 缩小字体 放大字体 收藏 微博 微信 分享 \n\n腾讯QQ QQ空间\n\n（来源：上观新闻）\n\n本周，我国天气上演群像戏。南方有大规模雨水，西北部有强降雪，中东部有较强冷空气，雾霾穿插来袭，还有台风赶着上，真是“你方唱罢我登场”。\n\n全国天气波澜起伏，上海则相对平稳。冷空气属于“过客”级别，来去匆匆。\n\n今天（3日），上海主打多云天气，冷空气遇上辐射降温，凉意拉满。市区早间最低温跌到13℃，郊区更冷，白天最高18℃。\n\n周三起气温直接反弹，最高不低于20℃，最低温16℃。晴好天气暂告一段落，会飘点短时小雨。', 'score': 0.91175514}]


In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma,FAISS
import dotenv
import os


dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("DASHSCOPE_BASE_URL")

#创建嵌入模型
embedding_model = DashScopeEmbeddings(model="text-embedding-v4")

#创建一个WebBaseLoader的实例，并将指定的文档加载
loader = WebBaseLoader(
    "https://zh.wikipedia.org/wiki/%E7%8C%AB"
)
docs = loader.load()

#创建文本拆分器，并拆分文档
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

documents = splitter.split_documents(docs)

vector = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

retriever = vector.as_retriever()

print(retriever.invoke("猫的特征")[0])


KeyboardInterrupt: 

In [13]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="wiki_search",
    description="搜索维基百科"
)

tools = [search,retriever_tool]

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="qwen-plus")

model_with_tools = model.bind_tools(tools)

messages = [HumanMessage(content="今天上海天气怎么样")]
response = model_with_tools.invoke(messages)
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '今天上海天气怎么样'}, 'id': 'call_16b9bdf8b16445abb59a17', 'type': 'tool_call'}]


In [15]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [32]:
from langchain.agents import create_agent_executor
from langgraph.prebuilt import create_agent_executor

agent = create_agent(model, tools, prompt)

agent_executor = create_agent_executor(agent, tools,verbose=True)



ImportError: cannot import name 'create_agent_executor' from 'langchain.agents' (/home/qixia/anaconda3/envs/llm/lib/python3.13/site-packages/langchain/agents/__init__.py)